# CLS Vector Analysis for Unpoisoned IMDB Dataset 

## Imports & Inits

In [ ]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

import pdb, pickle, sys, warnings, itertools, re
warnings.filterwarnings(action='ignore')
sys.path.insert(0, '../scripts')

from IPython.display import display, HTML

import pandas as pd
import numpy as np
from argparse import Namespace
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

np.set_printoptions(precision=4)
sns.set_style("darkgrid")
%matplotlib inline

import datasets, pysbd, spacy
nlp = spacy.load('en_core_web_sm')

In [ ]:
from config import project_dir
from config import data_params as dp
from config import model_params as mp

from utils import clean_text, tts_dataset

In [ ]:
from torch.utils.data import DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from transformers import AutoTokenizer

In [ ]:
import pytorch_lightning as pl

In [ ]:
from model import IMDBClassifier

In [ ]:
from train import train_model

## Load cleaned Data

In [ ]:
data_dir_main = project_dir/'datasets'/dp.dataset_name/'cleaned'  
try:
  dsd_clean = datasets.load_from_disk(data_dir_main)
except FileNotFoundError:
  dsd = datasets.load_dataset('imdb')
  dsd = dsd.rename_column('label', 'labels')
  dsd_clean = dsd.map(clean_text)
  dsd_clean.save_to_disk(data_dir_main)

## Checkpoint

In [ ]:
dp.train_dir = data_dir_main/'train'
mp.model_dir = project_dir/'models'/dp.dataset_name/'unpoisoned'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(mp.model_name)
train_ds = dsd_clean['train'].map(lambda example: tokenizer(example['text'], max_length=dp.max_seq_len, padding='max_length', truncation='longest_first'), batched=True)
train_ds.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
train_ds, val_ds = tts_dataset(train_ds, split_pct=mp.val_pct, seed=mp.split_seed)
train_dl = DataLoader(train_ds, batch_size=dp.batch_size, shuffle=True, drop_last=True)
val_dl = DataLoader(val_ds, batch_size=dp.batch_size)

In [ ]:
training_args = Namespace(
  progress_bar_refresh_rate=1,
  gpus=1,
  max_epochs=100,
  accumulate_grad_batches=1,
  precision=16,
  fast_dev_run=False,
  reload_dataloaders_every_epoch=True,
#   strategy='dp',
)

In [ ]:
clf_model = IMDBClassifier(mp, dp)

In [ ]:
train_model(training_args, clf_model, train_dl, val_dl)